# Imports

In [ ]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
from PIL import Image

import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '..', '_pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import interpolation_utils
import plot_utils
import data_utils

In [ ]:
plot_utils.set_rcParams()

In [ ]:
fig_num = os.getcwd().split('/')[-1][3:5]
print(fig_num)

In [ ]:
data_utils.make_dir('Figures')

# Plot COMSOL structure

In [ ]:
comsol_structure = Image.open(os.path.join('Images', 'comsol_structure.png'))

In [ ]:
def plot_comsol_structure(ax):
    
    ax.imshow(comsol_structure, extent=(0,100,0,100))
    
    ax.axis('off')
    ax.axis('equal')
    ax.text(3, 55, 'Ames\' medium:',
            horizontalalignment='left', verticalalignment='center')
    ax.text(3, 45, r'$\epsilon_{ames}$, $\sigma_{ames}$',
            horizontalalignment='left', verticalalignment='center')
    
    ax.text(3, 15, 'Retina: ' + r'$\epsilon_{retina}$, $\sigma_{retina}$',
            horizontalalignment='left', verticalalignment='center')
    
    ax.set_xlim((0, 110))
    ax.set_ylim((-7, 107))
    
    ax.text(100, 55, r'$h_{ames}$', verticalalignment='center')
    ax.text(100, 15, r'$h_{retina}$', verticalalignment='center')
    
    ax.text(5, -3, r'$r_{electrode}$', verticalalignment='center')
    ax.text(50, 102, r'$r_{retina}$', verticalalignment='center', horizontalalignment='center')

plot_comsol_structure(ax=plt.subplot(111))

# Plot Vext

In [ ]:
scale_factor = 1e6
Vm_scale_factor = 1e3

In [ ]:
def plot_Vext(ax, Vext_file, retina_h=105, ames_h=35, plot_r=None, el_r=None):
    
    Vext = pd.read_csv(Vext_file, comment="%", header=None, names=['x', 'r', 'V'])
    Vim  = interpolation_utils.interpolate_xyz2grid(Vext['x'], Vext['r'], Vext['V'])[2]
    # Mirror.
    Vim  = np.concatenate([np.fliplr(Vim),Vim[:,1:]], axis=1)*Vm_scale_factor
    
    ax.axis('off')
    
    w = Vext['r'].max()*scale_factor
    h = Vext['x'].max()*scale_factor
    
    heatmap = ax.imshow(
        Vim, origin='lower', cmap='Reds', extent=(-w, w, 0, h),
        vmin=0, interpolation='bicubic', vmax=100
    )
    
    if el_r is not None:
        ax.fill_between([-el_r, el_r], [-2,-2], [0, 0], color='k', lw=0, clip_on=False)
    
    ax.text(-plot_r+5, retina_h+35/2, 'Ames\' medium', verticalalignment='center')
    ax.text(-plot_r+5, retina_h-35/2, 'Retina', verticalalignment='center')
    
    ax.plot([30, 30], [25, 75], c='k')
    ax.text(33, 50, r"50 $\mu m$", verticalalignment='center')
    
    if plot_r is not None:
        ax.set_xlim((-plot_r, plot_r))
    else:
        ax.set_xlim(-w, w)
    ax.set_ylim(0, retina_h + ames_h)
    
    # make colorbar.
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    cax = make_axes_locatable(ax).append_axes("right", size="4%", pad=0.05)
    
    cb = plt.colorbar(heatmap, ax=ax, cax=cax, use_gridspec=True)
    cb.set_label(r'Voltage (mV)')   
    
    ax.axhline(retina_h, c='k', linestyle='--')

In [ ]:
def plot_comps(ax, comps_file, scale_comps):
    
    cell_comps = pd.read_csv(comps_file, names=['x', 'y', 'z'], delim_whitespace=True)
    
    ax.scatter(
        cell_comps['x']*scale_factor, cell_comps['z']*scale_factor,
        marker='o', color="k", alpha=1, s=scale_comps, facecolor='white', zorder=20, lw=scale_comps/7
    )
    ax.scatter(
        cell_comps['x'][0]*scale_factor, cell_comps['z'][0]*scale_factor,
        marker='o', color="k", alpha=1, s=scale_comps*4, facecolor='white', zorder=20, lw=scale_comps/7
    )
    
    neighbor_comps = get_neighbor_comps(cell_comps)
    
    for comp1 in range(cell_comps.shape[0]):
        x1 = cell_comps.iloc[comp1,:]['x']
        y1 = cell_comps.iloc[comp1,:]['y']
        z1 = cell_comps.iloc[comp1,:]['z']
        
        x2 = cell_comps.iloc[neighbor_comps[comp1],:]['x']
        y2 = cell_comps.iloc[neighbor_comps[comp1],:]['y']
        z2 = cell_comps.iloc[neighbor_comps[comp1],:]['z']
        
        ax.plot(
            np.array([x1, x2])*scale_factor, np.array([z1, z2])*scale_factor,
            color="dimgray", alpha=1, lw=scale_comps/4
        )

In [ ]:
def get_neighbor_comps(cell_comps):
    neighbor_comps = {}
    
    for comp1 in range(cell_comps.shape[0]):
        x1 = cell_comps.iloc[comp1,:]['x']
        y1 = cell_comps.iloc[comp1,:]['y']
        z1 = cell_comps.iloc[comp1,:]['z']
        
        d_min = 1000
        
        comp_d_min = None
            
        for comp2 in range(cell_comps.shape[0]):
            x2 = cell_comps.iloc[comp2,:]['x']
            y2 = cell_comps.iloc[comp2,:]['y']
            z2 = cell_comps.iloc[comp2,:]['z']
                
            if z1 < 6e-5 and z1 > 1.6e-5:
                if z2 > z1:
                    d12 = 5*(x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2
                else:
                    d12 = 1000
                
            else:
                d12 = (x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2
                
            if (d12 < d_min) and (comp1 != comp2):
                d_min      = d12
                comp_d_min = comp2
                
        if comp_d_min is not None:
            neighbor_comps[comp1] = comp_d_min
            
    return neighbor_comps

In [ ]:
def plot_Vext_and_comps(ax, scale_comps=5):

    Vext_file = os.path.join('..', 'Vext', 'Vext_simple.csv')
    comps_file = os.path.join('..', '..', 'COMSOL2retsim_interface', 'comsol_input', 'global', 'ON.csv')

    assert os.path.isfile(comps_file)

    plot_Vext(ax, Vext_file, retina_h=105, plot_r=85, el_r=15)
    plot_comps(ax, comps_file, scale_comps=scale_comps)

plt.figure(1,(3,3))
ax = plt.subplot(111)
plot_Vext_and_comps(ax, scale_comps=5)

In [ ]:
def plot_mesh(ax):
    ax.imshow(Image.open(os.path.join('Images', 'mesh.png')))
    ax.axis('off')
    ax.axis('equal')

In [ ]:
def plot_setup(ax):
    ax.imshow(Image.open(os.path.join('Images', 'ams_setup.png')))
    
    
    ax.text(70, 370, 'Ames\' medium', va='center', ha='left')
    ax.text(70, 480, 'Retina', va='center', ha='left')
    ax.text(70, 600, 'Micro-electrode array', va='center', ha='left')
    ax.text(410, 70, 'Flex MEA', va='center', ha='left')
    ax.text(1130, 230, 'Ag/AgCl', va='center', ha='right')
    ax.text(1330, 780, 'Ground', va='top', ha='right')
    
    ax.axis('off')
    ax.axis('equal')

In [ ]:
# Load images.
plot_mesh(plt.subplot(121))
plot_setup(plt.subplot(122))

# Make figure

In [ ]:
fig = plt.figure(figsize=(5.6, 1.5))

# Axis ratios.
sb_ny = 3
sb_nx = 12

ysb1=0.7
xsyb=0.3

# Plot.
ax1 = fig.add_axes((0.01,0.0,0.27,0.8))
plot_setup(ax1)

ax2 = fig.add_axes((xsyb,0.0,0.47-xsyb,0.85))
plot_mesh(ax2)

ax3 = fig.add_axes((0.48,0.0,0.20,0.9))
plot_comsol_structure(ax3)

ax4 = fig.add_axes((0.715,0.05,0.215,0.85))
plot_Vext_and_comps(ax4, scale_comps=4)

# Plot titles.
for ax, abc in zip([ax1, ax2, ax3, ax4], ["A", "B", "C", "D"]):
    ax = fig.add_axes((ax.get_position().bounds[0],0.75,0.1,0.1))
    ax.axis('off')
    ax.set_title(abc, loc='left', fontweight='bold', va='top', ha='left')
    
plt.savefig(f'../_figures/fig{fig_num}_Vext_explainatory.pdf', dpi=600)
plt.savefig("Figures/Vext_explainatory.png", dpi=600)
plt.show()

In [ ]:
# Good?
plt.imshow(Image.open("Figures/Vext_explainatory.png"))